In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from transformers import BertTokenizer

import warnings
warnings.filterwarnings("ignore")

In [20]:
# !pip install --upgrade transformers

In [21]:
from transformers import BertTokenizer
import tensorflow as tf

ImportError: Traceback (most recent call last):
  File "C:\Anaconda_latest\envs\english_language_learning_new\Lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 62, in <module>
    from tensorflow.python._pywrap_tensorflow_internal import *
ImportError: Module use of python38.dll conflicts with this version of Python.


Failed to load the native TensorFlow runtime.
See https://www.tensorflow.org/install/errors for some common causes and solutions.
If you need help, create an issue at https://github.com/tensorflow/tensorflow/issues and include the entire stack trace above this error message.

In [3]:
df = pd.read_csv('Datasets/Preprocessed_df.csv')

In [4]:
df.head()

,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions,preprocessed_text
0,0016926B079C,I think that students would benefit from learn...,3.5,3.5,3.0,3.0,4.0,3.0,i think that would benefit from learning at th...
1,0022683E9EA5,When a problem is a change you have to let it ...,2.5,2.5,3.0,2.0,2.0,2.5,when a problem is a change you have to let it ...
2,00299B378633,"Dear, Principal\n\nIf u change the school poli...",3.0,3.5,3.0,3.0,3.0,2.5,dear principal if u change the school policy o...
3,003885A45F42,The best time in life is when you become yours...,4.5,4.5,4.5,4.5,4.0,5.0,the best time in life is when you become yours...
4,0049B1DF5CCC,Small act of kindness can impact in other peop...,2.5,3.0,3.0,3.0,2.5,2.5,small act of kindness can impact in other peop...


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3911 entries, 0 to 3910
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   text_id            3911 non-null   object 
 1   full_text          3911 non-null   object 
 2   cohesion           3911 non-null   float64
 3   syntax             3911 non-null   float64
 4   vocabulary         3911 non-null   float64
 5   phraseology        3911 non-null   float64
 6   grammar            3911 non-null   float64
 7   conventions        3911 non-null   float64
 8   preprocessed_text  3911 non-null   object 
dtypes: float64(6), object(3)
memory usage: 275.1+ KB


In [6]:
df.drop(['text_id', 'full_text'], axis = 1, inplace = True)

In [7]:
df.head()

,cohesion,syntax,vocabulary,phraseology,grammar,conventions,preprocessed_text
0,3.5,3.5,3.0,3.0,4.0,3.0,i think that would benefit from learning at th...
1,2.5,2.5,3.0,2.0,2.0,2.5,when a problem is a change you have to let it ...
2,3.0,3.5,3.0,3.0,3.0,2.5,dear principal if u change the school policy o...
3,4.5,4.5,4.5,4.5,4.0,5.0,the best time in life is when you become yours...
4,2.5,3.0,3.0,3.0,2.5,2.5,small act of kindness can impact in other peop...


### BERT Architecture

In [8]:
# Split the data into train and test sets
train_text, test_text, train_score, test_score = train_test_split(df["preprocessed_text"], df["cohesion"], test_size=0.2, random_state=42)

# Load the pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

# Tokenize the text
train_encodings = tokenizer(list(train_text), truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(list(test_text), truncation=True, padding=True, max_length=128)

# Convert the data to TensorFlow tensors
train_data = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_score
)).batch(32)
test_data = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    test_score
)).batch(32)

# Define the regression model using the BERT encoder
input_layer = tf.keras.layers.Input(shape=(128,), dtype=tf.int32, name='input_layer')
embedding_layer = bert_model(input_layer)[1]
output_layer = tf.keras.layers.Dense(1, activation="linear")(embedding_layer)
model = tf.keras.models.Model(inputs=input_layer, outputs=output_layer)
model.compile(optimizer="adam", loss="mean_squared_error")

# Train the model
model.fit(train_data, epochs=10, validation_data=test_data)

# Evaluate the model on the test set
predictions = model.predict(test_data)
mse = mean_squared_error(test_score, predictions)
print("Mean Squared Error:", mse)


NameError: name 'TFBertModel' is not defined

In [17]:
import sys

print(sys.version)

3.10.9 | packaged by Anaconda, Inc. | (main, Mar  8 2023, 10:42:25) [MSC v.1916 64 bit (AMD64)]
